In [ ]:
"""Accruals

Demonstration of how to model accruals in LUSID

Attributes
----------
transaction configuration
cocoon
holdings
"""

# Accruals

This notebook demonstrates how accruals can be modelled in LUSID. We show how we can book a management fee that is due to be paid at the end of the month and track the accrual separately from the main cash balance. At the end of the month we see the fee being deducted from the cash balance.

In [1]:
# Import lusid specific packages
import lusid
import lusid.models as models
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon import identify_cash_items
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.utilities import create_scope_id

import os
import pandas as pd
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.5.4403.0


# Transaction configuration

Define a transaction configuration for the accrual, here we set the accrual account increase which will then be added to the cash balance on settlement.

In [2]:
transaction_type_scope = create_scope_id()

# Before setting the transaction type config, some side definitions are needed in the non-default transaction type scope.
# If working in the default scope, these side definitions are already inserted so this step can be skipped.
default_side_definitions = api_factory.build(lusid.api.TransactionConfigurationApi).list_side_definitions()

def mapSide(side_definition):
    return {"side": side_definition.side, "sideRequest": lusid.models.SideDefinitionRequest(security = side_definition.security, currency = side_definition.currency, rate = side_definition.rate, units = side_definition.units, amount = side_definition.amount, notional_amount= side_definition.notional_amount)}

set_side_definitions_request = [mapSide(side_definition) for side_definition in default_side_definitions.values]
api_factory.build(lusid.api.TransactionConfigurationApi).set_side_definitions(set_side_definitions_request, scope = transaction_type_scope)

# Add the default transaction types to the new scope
default_scope_txn_types_response = api_factory.build(lusid.api.TransactionConfigurationApi).list_transaction_types()
default_scope_default_source_transaction_types = default_scope_txn_types_response["default"]
txn_type_requests = [lusid.models.TransactionTypeRequest(txn_type.aliases, [lusid.models.TransactionTypeMovement(m.movement_types, m.side, m.direction, m.properties, m.mappings, m.name) for m in txn_type.movements], txn_type.properties) for txn_type in default_scope_default_source_transaction_types]
api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type_source(
    source="default",
    scope=transaction_type_scope,
    transaction_type_request=txn_type_requests)

# Set the new transaction type
accrual_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type="MgmtFee",
            description="Management fee accrual",
            transaction_class="Basic",
            transaction_roles="Longer"
        ),
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types="CashAccrual",
            side="Side2",
            direction=-1
        )
    ]
)
api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type("default", "MgmtFee", accrual_config, scope=transaction_type_scope)

Create a scope and portfolio code

In [3]:
scope = "accruals-demo"
portfolio_code = "EQUITY_UK"

In this portfolio we add a cash injection and book a transaction for a stock purchase. We also book an accrual for a management fee that is paid at the end of the month

In [4]:
df = pd.read_csv("data/accruals/equity_transactions.csv")
df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,01/04/2020,03/04/2020,120000,5,600000,GBP,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,01/04/2020,03/04/2020,1000000,1,1000000,GBP,GBP
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_002,MgmtFee,01/04/2020,30/04/2020,600,1,600,GBP,GBP


In [5]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
    "cash_flag": {
        "cash_identifiers": {
            "cash_transactions" : ["GBP"]
        },
        "implicit": "currency"
    }
}

In [6]:
instr_df, mapping = identify_cash_items(
    dataframe = df.copy(),
    mappings = instrument_mapping,
    file_type = "instruments",
    remove_cash_items = True
)

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])


,success,failed,errors
0,1,0,0


## Create portfolio

In [7]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [8]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

# Call LUSID to update the transaction type scope of your portfolio 
patch_document = [
    {
        "value": transaction_type_scope,
        "path": "/transactiontypescope",
        "op": "add"
    }
]
patch_response = api_factory.build(lusid.api.TransactionPortfoliosApi).patch_portfolio_details(
    scope=scope,
    code=portfolio_code,
    operation=patch_document)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


# Upload transactions

In [9]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions"
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [10]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


# Get holdings

In [11]:
# transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
transaction_portfolio_api = lusid.api.TransactionPortfoliosApi(api_factory.build(lusid.api.TransactionPortfoliosApi))

def get_holdings(effective_at):
    response = transaction_portfolio_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=effective_at,
        property_keys=["Instrument/default/Name"]
    )

    holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
    display(holdings_df)

Getting the holdings before settlement shows:
* the holding in the purchased stock (P)
* the committed cash against the purchase (C)
* the received cash injection (R)
* the accrual from the fee (A)

In [12]:
get_holdings("2020-04-01")

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/ClientInternal,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.instrument_identifiers.Instrument/default/Currency,transaction.properties
0,LUID_JF8O2838,{},Aviva,accruals-demo/EQUITY_UK,P,120000.0,0.0,600000.0,GBP,0.0,GBP,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,C,-600000.0,0.0,-600000.0,GBP,0.0,GBP,trd_0001,Buy,EQ_1234,LUID_JF8O2838,2020-04-01 00:00:00+00:00,2020-04-03 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,Transaction/default/ResultantHolding,120000.0,NaN,NaN
2,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,A,1000000.0,0.0,1000000.0,GBP,0.0,GBP,cash_001,FundsIn,NaN,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-03 00:00:00+00:00,1000000.0,1.0,Price,1000000.0,GBP,1.0,GBP,NaN,NaN,GBP,{}
3,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,A,600.0,0.0,600.0,GBP,0.0,GBP,cash_002,MgmtFee,NaN,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-30 00:00:00+00:00,600.0,1.0,Price,600.0,GBP,1.0,GBP,NaN,NaN,GBP,{}


After the stock has settled we can see the cash balance updated and the fee accrual still there.

In [13]:
get_holdings("2020-04-06")

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Currency,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties
0,LUID_JF8O2838,{},Aviva,accruals-demo/EQUITY_UK,P,120000.0,120000.0,600000.0,GBP,0.0,GBP,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,B,400000.0,400000.0,400000.0,GBP,0.0,GBP,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,A,600.0,0.0,600.0,GBP,0.0,GBP,cash_002,MgmtFee,GBP,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-30 00:00:00+00:00,600.0,1.0,Price,600.0,GBP,1.0,GBP,{}


At the end of the month the accrual is paid out and the cash balance updated

In [14]:
get_holdings("2020-04-30")


,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_JF8O2838,{},Aviva,accruals-demo/EQUITY_UK,P,120000.0,120000.0,600000.0,GBP,0.0,GBP
1,CCY_GBP,{},CCY_GBP,accruals-demo/EQUITY_UK,B,400600.0,400600.0,400600.0,GBP,0.0,GBP
